import subprocess

cmd = "mpiexec -n 4 ./mstm-ampmat-mpi mstm-v4.0-agg.inp"

for i in range(1):
    try:
        res = subprocess.check_call(cmd.split())
    except:
        print("Error.")


In [ ]:
import glob
import os
import numpy as np
import subprocess

In [ ]:
agg_num=0

kf=0.950
meanRp=0.020
rstdRp=0.10

agg_folderpath= "./aggregate_ptsa_files/"
mstm_output_folderpath= "./mstm_out_files/"

wl0= 0.6328 # vaccuum wavelength [um]
m_med= 1.33154 # medium refractive index @ wl0
wavenumber_med= m_med*2*np.pi/wl0 #[um-1]


# number of monomers
Np_min= 20
Np_max= 16000
num_Np= 30 # Number of aggregate Np between Np_min and Np_max
Np_arr= np.floor(np.logspace(start= np.log10(Np_min), stop= np.log10(Np_max), num=num_Np)).astype(int)

min_real_rel_ref_index = 1.02
max_real_rel_ref_index = 2.421
step_real_rel_ref_index = 0.07
real_rel_ref_index_arr= np.arange(start= min_real_rel_ref_index, stop= max_real_rel_ref_index, step=step_real_rel_ref_index)


min_imag_rel_ref_index = 0.0
max_imag_rel_ref_index = 1.401
step_imag_rel_ref_index = 0.07
imag_rel_ref_index_arr= np.arange(start= min_imag_rel_ref_index, stop= max_imag_rel_ref_index, step= step_imag_rel_ref_index)

# fractal dimension
Df_min= 2.35 #(default)
Df_max= 2.95 #(default)
num_Df= 13
Df_arr= np.linspace(start=Df_min, stop=Df_max, num=num_Df)

for Df in Df_arr:
    for Np in Np_arr:
        agg_filepath= "agg_num={:01d}_k={:5.3f}_Df={:4.2f}_meanRp={:5.3f}um_rstdRp={:4.2f}_Np={:05d}_".format(agg_num, kf, Df, meanRp, rstdRp, Np)
        input_spheres_filepath= glob.glob(agg_folderpath+agg_filepath+'*')[0]
        for real_rel_ref_index in real_rel_ref_index_arr:
            #if Np == 12706 and real_rel_ref_index < 1.37:
            #    continue
            for imag_rel_ref_index in imag_rel_ref_index_arr:
                mstm_output_filename= "mstm_res_wl0={:6.4f}_m_med={:7.5f}_".format(wl0, m_med) + os.path.splitext(os.path.basename(input_spheres_filepath))[0] + "_real_ref_index={:4.2f}_imag_ref_index={:4.2f}".format(real_rel_ref_index, imag_rel_ref_index) + '.out'
                mstm_output_filepath= mstm_output_folderpath + mstm_output_filename
                # make an input file for mstm run
                mstm_input_filename= "mstm-v4.0-agg{:01d}".format(agg_num) + "-batch.inp"
                with open(mstm_input_filename, 'w') as f:
                    #line= "{:13.7e}  {:13.7e}  {:13.7e}  {:13.7e}".format(xp[ip,0],xp[ip,1],xp[ip,2],rp[ip])
                    f.write("output_file" + '\n')
                    f.write(mstm_output_filepath + '\n')
                    f.write("number_spheres" + '\n')
                    f.write("{:d}".format(Np) + '\n')
                    f.write("sphere_data_input_file" + '\n')
                    f.write(input_spheres_filepath + '\n')
                    f.write("print_sphere_data" + '\n')
                    f.write('f' + '\n')
                    f.write("random_orientation" + '\n')
                    f.write('f' + '\n')
                    f.write("incident_beta_deg" + '\n')
                    f.write('0.d0' + '\n')
                    f.write("incident_alpha_deg" + '\n')
                    f.write('0.d0' + '\n')
                    f.write("calculate_scattering_matrix" + '\n')
                    f.write('t' + '\n')
                    f.write("scattering_map_model" + '\n')
                    f.write('0' + '\n')
                    f.write("scattering_map_increment" + '\n')
                    f.write('90' + '\n')
                    f.write("fft_translation_option" + '\n')
                    f.write('t' + '\n')
                    f.write("length_scale_factor" + '\n')
                    f.write("{:f}".format(wavenumber_med) + '\n')
                    f.write("ref_index_scale_factor" + '\n')
                    f.write("({:4.2f}, {:4.2f})".format(real_rel_ref_index, imag_rel_ref_index) + '\n')
                    f.write("end_of_options" + '\n')
                
                if Np > 1000 :
                    num_CPU_core= 1
                else:
                    num_CPU_core= 1
            
                cmd = "mpiexec -n " + "{:d}".format(num_CPU_core) + " ./mstm-ampmat_fb-mpi " + mstm_input_filename
                try:
                    res = subprocess.check_call(cmd.split())
                except:
                    print("Error.")
